In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
path = "/home/fiend/Documents"

In [ ]:
df = pd.read_csv(f'{path}/processed_df.csv', parse_dates=True)

In [ ]:
period_start = pd.to_datetime('2023-07-01')
period_end = pd.to_datetime('2024-06-30')
# outcome = 'concluded'

# # Convert activity_date to datetime
df['activity_date'] = pd.to_datetime(df['activity_date'], errors='coerce')

kisumu_df = df[(df['court'] =='Kisumu') &
    (df['activity_date'] >= period_start) &
    (df['activity_date'] <= period_end)
]

In [ ]:
judges = ['Aburili, Roselyne', 'Saida,shariff Mwanaisha']
kisumu_df = kisumu_df[kisumu_df['judge_1'].isin(judges)].copy()

In [ ]:
kisumu_df = kisumu_df.copy(deep=True)

In [ ]:
kisumu_df.groupby('case_type')['concluded'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_df.columns

In [ ]:
# determine Judicial Review cases resolved within 180 days
kisumu_df['concluded_within_180_days'] = kisumu_df['concluded'] & (kisumu_df['age'] <= 180)


In [ ]:
# percentage of judicial review cases resolved within 180 days
percentage_resolved_within_180_days = (
	kisumu_df[(kisumu_df['case_type'] == 'Judicial Review') & (kisumu_df['concluded_within_180_days'])]['concluded'].sum() /
	kisumu_df[kisumu_df['case_type'] == 'Judicial Review']['concluded'].sum()
) * 100


In [ ]:
kisumu_df[(kisumu_df['case_type'] == 'Judicial Review') & (kisumu_df['concluded_within_180_days'])]['concluded'].sum()

In [ ]:
kisumu_df[kisumu_df['case_type'] == 'Judicial Review']['concluded'].sum()

In [ ]:
percentage_resolved_within_180_days

In [ ]:
kisumu_df.loc[:, 'appearance_count'] = kisumu_df.groupby('case_number')['case_number'].transform('count')

In [ ]:
kisumu_df['resolved'] = kisumu_df.groupby('case_number')['concluded'].transform('max')


In [ ]:
kisumu_resolved = kisumu_df[kisumu_df['resolved'] >=1].copy()

In [ ]:
kisumu_resolved.loc[:, 'ruling_delivered'] = (kisumu_resolved['outcome'] == 'Ruling Delivered- Case Closed').astype(int)

In [ ]:
kisumu_resolved.loc[:, 'ruling_delivered_max'] = kisumu_resolved.groupby('case_number')['ruling_delivered'].transform('max')

In [ ]:
kisumu_ruling_df = kisumu_resolved[kisumu_resolved['ruling_delivered_max'] >= 1].copy()

In [ ]:
# ruling shceduled Ruling Date Given
kisumu_ruling_df.loc[:, 'ruling_scheduled'] = (kisumu_ruling_df['outcome'] == 'Ruling Date Given').astype(int)
kisumu_ruling_df.loc[:, 'ruling_scheduled_max'] = kisumu_ruling_df.groupby('case_number')['ruling_scheduled'].transform('max')

In [ ]:
ruling_not_scheduled = kisumu_ruling_df[kisumu_ruling_df['ruling_scheduled_max'] == 0]

In [ ]:
kisumu_df.query('case_number == "Kisumu/HCCRC/22/2020"')

In [ ]:
ruling_not_scheduled

In [ ]:
resolved_cases = kisumu_df.pivot_table(index='case_type', columns='productivity', values='concluded', aggfunc='sum', fill_value=0)

In [ ]:
resolved_cases_not_scheduled = ruling_not_scheduled.pivot_table(index='case_type', columns='productivity', values='concluded', aggfunc='sum', fill_value=0)

In [ ]:
# 
merged = pd.merge(resolved_cases, resolved_cases_not_scheduled, on='case_type', suffixes=('_resolved', '_not_scheduled'))

In [ ]:
merged

In [ ]:
ruling_not_scheduled.groupby('case_type')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
ruling_not_scheduled[(ruling_not_scheduled['appearance_count'] >= 1) & (ruling_not_scheduled['appearance_count'] <= 2)].groupby('case_type')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_ruling_df.groupby('judge_1')['ruling_delivered'].sum().reset_index(name='total').sort_values(by='total', ascending=False)

In [ ]:
kisumu_ruling_df.groupby('case_type')['ruling_scheduled'].sum().reset_index(name='total').sort_values(by='total', ascending=False)